In [5]:
import subprocess
import time

def esegui_programma_c(n, percorso_programma,args,mpi):
    
    if mpi:
        comando = ['mpiexec','-n','4',percorso_programma] + args
    else:
        comando = [percorso_programma] + args

    for i in range(n):
        # Esegui il programma C
        risultato = subprocess.run(comando, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


def write_efficiency(filename,out):
    with open(filename, "r") as f:
        content = f.readlines()
        for line in content:
            tmp = line.split()
            print("tmp: ",tmp,"\n")
            if tmp == []:
                continue
            nprocesses = filename.split('.')[0][-1]
            print("nprocesses: ",nprocesses,"\n")
            efficiency = float(tmp[1])/float(nprocesses)
            print("efficiency: ",efficiency,"\n")
            with open(out, "a") as f:
                f.write(tmp[0] + ": " + str(efficiency) + "\n")
    return


def write_results(listofiles):
    # open a file,read the content and split with endline
    with open("results.csv", "w") as f:
        pass
    serialbruteforce2000 = 0
    serialbarneshut2000 = 0
    serialbruteforce3000 = 0
    serialbarneshut3000 = 0
    paralleltimes2000 = []
    paralleltimes3000 = []

    for filenames in listofiles:
        listoftimes = []
        with open(filenames, "r") as f:
            content = f.readlines()
            for line in content:
                # read every line and split with space
                line = line.split()
                usecase,time = line[0],line[-1]
                listoftimes.append((usecase,float(time)))
            listoftimes.sort(key=lambda x: x[0])
            twothousand = listoftimes[0:5]
            threethousand = listoftimes[5:]

            with open("results.csv", "a") as f:
                f.write(filenames + " [2000] : " + str(sum([x[1] for x in twothousand])/len(twothousand)) + "\n\n")
                f.write(filenames + " [3000]: " + str(sum([x[1] for x in threethousand])/len(threethousand)) + "\n\n")
    with open("results.csv","r") as f:
        content = f.readlines()
        for line in content:
            tmp = line.split()
            if tmp == []:
                continue
            if tmp[0] == "serial-bh-times.csv" and tmp[1] == "[2000]":
                serialbarneshut2000 = float(tmp[-1])
            elif tmp[0] == "serial-nbodies-times.csv" and tmp[1] == "[2000]":
                serialbruteforce2000 = float(tmp[-1])
            elif tmp[0] == "serial-bh-times.csv" and tmp[1] == "[3000]:":
                serialbarneshut3000 = float(tmp[-1])
            elif tmp[0] == "serial-nbodies-times.csv" and tmp[1] == "[3000]:":
                serialbruteforce3000 = float(tmp[-1])
            elif tmp[1] == "[2000]":
                paralleltimes2000.append((tmp[0],float(tmp[-1])))
            elif tmp[1] == "[3000]:":
                paralleltimes3000.append((tmp[0],float(tmp[-1])))

    # ------ vvv implicit destination file --- vvv
    with open("speedups4.csv","w") as s: 
        # -------^^^^----------------
        for elements in paralleltimes2000:
            library,algorithm = elements[0].split("-")[0],elements[0].split("-")[1]
            print(library,algorithm)
            if library == "mpi" and algorithm == "bh":
                time = elements[1]
                speedup = serialbarneshut2000/time
                s.write("MPI-BH-2000: " + str(speedup) + "\n")
            elif library == "mpi" and algorithm == "parallel":
                time = elements[1]
                speedup = serialbruteforce2000/time
                s.write("MPI-Parallel-2000: " + str(speedup) + "\n")
            elif library == "pthread" and algorithm == "bh":
                time = elements[1]
                speedup = serialbarneshut2000/time
                s.write("Pthread-BH-2000: " + str(speedup) + "\n")
            elif library == "pthread" and algorithm == "parallel":
                time = elements[1]
                speedup = serialbruteforce2000/time
                s.write("Pthread-Parallel-2000: " + str(speedup) + "\n")
        s.write("\n\n\n")

            
            
        for elements in paralleltimes3000:
            library,algorithm = elements[0].split("-")[0],elements[0].split("-")[1]
            print(library,algorithm)
            if library == "mpi" and algorithm == "bh":
                time = elements[1]
                speedup = serialbarneshut3000/time
                s.write("MPI-BH-3000: " + str(speedup) + "\n")
            elif library == "mpi" and algorithm == "parallel":
                time = elements[1]
                speedup = serialbruteforce3000/time
                s.write("MPI-Parallel-3000: " + str(speedup) + "\n")
            elif library == "pthread" and algorithm == "bh":
                time = elements[1]
                speedup = serialbarneshut3000/time
                s.write("Pthread-BH-3000: " + str(speedup) + "\n")
            elif library == "pthread" and algorithm == "parallel":
                time = elements[1]
                speedup = serialbruteforce3000/time
                s.write("Pthread-Parallel-3000: " + str(speedup) + "\n")
    return
import sys
n = 3 # Numero di esecuzioni
args = ['-t', '1000', '-S', 'random', '-n', '2000']  

print("Eseguendo i test con 2000 corpi")

with open("mpi-bh-times.csv", "w") as f:
    pass

with open("mpi-parallel-times.csv", "w") as f:
    pass

with open("pthread-bh-times.csv", "w") as f:
    pass

with open("pthread-parallel-times.csv", "w") as f:
    pass

with open("serial-bh-times.csv", "w") as f:
    pass
with open("serial-nbodies-times.csv", "w") as f:
    pass



print("Eseguendo i test serial_nbodies\n")
percorso_programma = '../serial_nbodies/main' 
esegui_programma_c(n, percorso_programma,args,False)

print("Eseguendo i test mpi-nbodies\n")
percorso_programma = '../mpi_parallel_nbodies/main'
esegui_programma_c(n,percorso_programma,args,True)

print("Eseguendo i test pthread-nbodies\n")
percorso_programma = '../pthread_parallel_nbodies/main'
esegui_programma_c(n,percorso_programma,args,False)

print("Eseguendo i test serial-bh\n")
percorso_programma = '../BarnesHut_serial/main'
esegui_programma_c(n,percorso_programma,args,False)

print("Eseguendo i test mpi-bh\n")
percorso_programma = '../mpi_BarnesHut/main'
esegui_programma_c(n,percorso_programma,args,True)


print("Eseguendo i test pthread-bh\n")
percorso_programma = '../pthread_BarnesHut/main'
esegui_programma_c(n,percorso_programma,args,False)

args = ['-t', '1000', '-S', 'random', '-n', '3000']  

print("eseguendo i test con 3000 corpi\n")

print("Eseguendo i test serial-nbodies\n")
percorso_programma = '../serial_nbodies/main' 
esegui_programma_c(n, percorso_programma,args,False)

print("Eseguendo i test serial-bh\n")
percorso_programma = '../BarnesHut_serial/main'
esegui_programma_c(n,percorso_programma,args,False)

print("Eseguendo i test mpi-nbodies\n")
percorso_programma = '../mpi_parallel_nbodies/main'
esegui_programma_c(n,percorso_programma,args,True)

print("Eseguendo i test mpi-bh\n")
percorso_programma = '../mpi_BarnesHut/main'
esegui_programma_c(n,percorso_programma,args,True)

print("Eseguendo i test pthread-nbodies\n")
percorso_programma = '../pthread_parallel_nbodies/main'
esegui_programma_c(n,percorso_programma,args,False)

print("Eseguendo i test pthread-bh\n")
percorso_programma = '../pthread_BarnesHut/main'
esegui_programma_c(n,percorso_programma,args,False)


write_results(["mpi-bh-times.csv", "mpi-parallel-times.csv", "pthread-bh-times.csv", "pthread-parallel-times.csv", "serial-bh-times.csv", "serial-nbodies-times.csv"])
write_efficiency("speedups2.csv","efficiencies2.csv")
write_efficiency("speedups4.csv","efficiencies4.csv")


tmp:  ['MPI-BH-2000:', '1.802275159782236'] 

nprocesses:  2 

efficiency:  0.901137579891118 

tmp:  ['MPI-Parallel-2000:', '1.9192008430794236'] 

nprocesses:  2 

efficiency:  0.9596004215397118 

tmp:  ['Pthread-BH-2000:', '1.8644269680509558'] 

nprocesses:  2 

efficiency:  0.9322134840254779 

tmp:  ['Pthread-Parallel-2000:', '1.9446077806467597'] 

nprocesses:  2 

efficiency:  0.9723038903233798 

tmp:  [] 

tmp:  [] 

tmp:  ['MPI-BH-3000:', '1.7401404846145616'] 

nprocesses:  2 

efficiency:  0.8700702423072808 

tmp:  ['MPI-Parallel-3000:', '1.9532525101748326'] 

nprocesses:  2 

efficiency:  0.9766262550874163 

tmp:  ['Pthread-BH-3000:', '1.7925239115304739'] 

nprocesses:  2 

efficiency:  0.8962619557652369 

tmp:  ['Pthread-Parallel-3000:', '1.930493907923825'] 

nprocesses:  2 

efficiency:  0.9652469539619125 

tmp:  ['MPI-BH-2000:', '2.101057211243509'] 

nprocesses:  4 

efficiency:  0.5252643028108772 

tmp:  ['MPI-Parallel-2000:', '3.6564099643486196'] 

nproces